In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install pillow requests

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import urllib.request
from PIL import Image
import io
from tensorflow.keras.regularizers import l2

2025-09-17 12:03:47.714106: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758110628.097333      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758110628.207731      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Checking GPU status

In [3]:
def check_gpu_status():
    """
    Checks if a GPU is available and prints the status.
    """
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Set memory growth to avoid allocating all GPU memory at once
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print(f"Found {len(gpus)} GPU(s): {gpus}")
            return True
        except RuntimeError as e:
            print(e)
            return False
    else:
        print("No GPU found. Training will run on CPU.")
        return False

print(check_gpu_status())

Found 2 GPU(s): [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
True


CIFAR 100 dataset has 20 superclasses each containing 5 subclasses.

So now I will be training 20 different models on each of the superclasses and then train the gating model for Mixture Of Experts.

So now for the 20 different models, I will be creating seperate training data and labels.

In [ ]:
#This dictionary maps 100 fine grained classes to 20 coearse grained superclasses.

fine_to_coarse_labels = {}

for i in range(0,100):
    n = i // 5
    fine_to_coarse_labels[i] = n

    
superclass_names = [
    "aquatic mammals", "fish", "flowers", "food containers", "fruit and vegetables",
    "household electrical devices", "household furniture", "insects", "large carnivores",
    "large man-made outdoor things", "large natural outdoor scenes",
    "large omnivores and herbivores", "medium-sized mammals", "non-insect invertebrates",
    "people", "reptiles", "small mammals", "trees", "vehicles 1", "vehicles 2"
]

def get_superclass_data():
    (x_train_fine, y_train_fine), (x_test_fine, y_test_fine) = tf.keras.datasets.cifar100.load_data(label_mode='fine')
    superclass_data = {}
    test_data = {}

    for name in superclass_names:
        superclass_data[name] = {'images':[], 'labels':[]}
        test_data[name] = {'images':[], 'labels':[]}

    for i in range(len(x_train_fine)):
        fine_label = y_train_fine[i][0]
        coarse_label = fine_to_coarse_labels[fine_label]
        superclass_name = superclass_names[coarse_label]

        superclass_data[superclass_name]['images'].append(x_train_fine[i])
        superclass_data[superclass_name]['labels'].append(fine_label)

    for i in range(len(x_test_fine)):
        fine_label = y_test_fine[i][0]
        coarse_label = fine_to_coarse_labels[fine_label]
        testclass_name = superclass_names[coarse_label]

        test_data[testclass_name]['images'].append(x_test_fine[i])
        test_data[testclass_name]['labels'].append(fine_label)

    for name in superclass_names:
        images = np.array(superclass_data[name]['images'])
        labels = np.array(superclass_data[name]['labels'])
        superclass_data[name] = (images,labels)

    for name in superclass_names:
        images = np.array(test_data[name]['images'])
        labels = np.array(test_data[name]['labels'])
        test_data[name] = (images,labels)
    
    return superclass_data, test_data

In [ ]:
superclass_data, testclass_data = get_superclass_data()

In [ ]:
all_data = list(superclass_data.values())
all_test_data = list(testclass_data.values())

In [ ]:
all_images = []
all_labels = []

for tup in all_data:
    tup_l = list(tup)
    all_images.append(tup_l[0])
    all_labels.append(tup_l[1])


all_test_images = []
all_test_labels = []

for tup in all_test_data:
    tup_l = list(tup)
    all_test_images.append(tup_l[0])
    all_test_labels.append(tup_l[1])
    

Now building and training 20 different models each for a different superclass.

In [ ]:
#Initially the model had no dropout layer in it due to which we were suffering from model overfitting and hence adding a dropout layer in the model

def build_model(num_classes):
    model = Sequential([
       # First Convolutional Block
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
        MaxPooling2D((2, 2)),
        # Second Convolutional Block
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        # Third Convolutional Block
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        # Flatten the feature maps to feed into the dense layers
        Flatten(),
        #Dropout later to prevent overfitting
        Dropout(0.6),
        # Fully Connected Layers
        Dense(128, activation='relu'),
        # Output layer with 'num_classes' neurons for classification
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])

    return model

def train_single_superclass_model(x_train_superclass,y_train_superclass,x_test_superclass,y_test_superclass,superclass_name,n_epochs):
    y_train_flat = y_train_superclass.flatten()
    y_test_flat = y_test_superclass.flatten()
    unique_labels = np.unique(y_train_flat)
    num_classes = len(unique_labels)
    label_map = {label: i for i, label in enumerate(unique_labels)}
    y_train_superclass_list = y_train_superclass.tolist()
    y_test_superclass_list = y_test_superclass.tolist()
    y_train_mapped = np.array([label_map[label] for label in y_train_flat])
    y_test_mapped = np.array([label_map[label] for label in y_test_flat])

    print(f"\n--- Building and training model for superclass: {superclass_name} ---")
    print(f"Number of unique fine classes: {num_classes}")
    is_gpu_available = check_gpu_status()
    if is_gpu_available:
        with tf.device('/GPU:0'):
            model = build_model(num_classes)
            model.fit(x_train_superclass, y_train_mapped, epochs = n_epochs,batch_size = 32, verbose = 0)
    else:
        model = build_model(num_classes)
        model.fit(x_train_superclass, y_train_mapped, epochs = n_epochs,batch_size = 32, verbose = 0)
    
    print(f"Training for {superclass_name} complete.")

    print(f"\n--- Evaluating model for superclass: {superclass_name} ---")
    loss, accuracy = model.evaluate(x_test_superclass, y_test_mapped, verbose=1)
    
    print(f"Test Loss: {loss:.4f}")
    print(f"Test Accuracy: {accuracy:.4f}")
    return model

In [ ]:
aq_m_x_train = all_images[0]
fish_x_train = all_images[1]
flwr_x_train = all_images[2]
fooc_x_train = all_images[3]
fav_x_train = all_images[4]
hed_x_train = all_images[5]
hf_x_train = all_images[6]
ins_x_train = all_images[7]
lc_x_train = all_images[8]
lmot_x_train = all_images[9]
lnos_x_train = all_images[10]
loah_x_train = all_images[11]
msm_x_train = all_images[12]
nii_x_train = all_images[13]
peo_x_train = all_images[14]
rep_x_train = all_images[15]
sm_x_train = all_images[16]
trees_x_train = all_images[17]
v1_x_train = all_images[18]
v2_x_train = all_images[19]


aq_m_y_train = all_labels[0].reshape(-1,1)
fish_y_train = all_labels[1].reshape(-1,1)
flwr_y_train = all_labels[2].reshape(-1,1)
fooc_y_train = all_labels[3].reshape(-1,1)
fav_y_train = all_labels[4].reshape(-1,1)
hed_y_train = all_labels[5].reshape(-1,1)
hf_y_train = all_labels[6].reshape(-1,1)
ins_y_train = all_labels[7].reshape(-1,1)
lc_y_train = all_labels[8].reshape(-1,1)
lmot_y_train = all_labels[9].reshape(-1,1)
lnos_y_train = all_labels[10].reshape(-1,1)
loah_y_train = all_labels[11].reshape(-1,1)
msm_y_train = all_labels[12].reshape(-1,1)
nii_y_train = all_labels[13].reshape(-1,1)
peo_y_train = all_labels[14].reshape(-1,1)
rep_y_train = all_labels[15].reshape(-1,1)
sm_y_train = all_labels[16].reshape(-1,1)
trees_y_train = all_labels[17].reshape(-1,1)
v1_y_train = all_labels[18].reshape(-1,1)
v2_y_train = all_labels[19].reshape(-1,1)


#For testing data
aq_m_x_test = all_test_images[0]
fish_x_test = all_test_images[1]
flwr_x_test = all_test_images[2]
fooc_x_test = all_test_images[3]
fav_x_test = all_test_images[4]
hed_x_test = all_test_images[5]
hf_x_test = all_test_images[6]
ins_x_test = all_test_images[7]
lc_x_test = all_test_images[8]
lmot_x_test = all_test_images[9]
lnos_x_test = all_test_images[10]
loah_x_test = all_test_images[11]
msm_x_test = all_test_images[12]
nii_x_test = all_test_images[13]
peo_x_test = all_test_images[14]
rep_x_test = all_test_images[15]
sm_x_test = all_test_images[16]
trees_x_test = all_test_images[17]
v1_x_test = all_test_images[18]
v2_x_test = all_test_images[19]


aq_m_y_test = all_test_labels[0].reshape(-1,1)
fish_y_test = all_test_labels[1].reshape(-1,1)
flwr_y_test = all_test_labels[2].reshape(-1,1)
fooc_y_test = all_test_labels[3].reshape(-1,1)
fav_y_test = all_test_labels[4].reshape(-1,1)
hed_y_test = all_test_labels[5].reshape(-1,1)
hf_y_test = all_test_labels[6].reshape(-1,1)
ins_y_test = all_test_labels[7].reshape(-1,1)
lc_y_test = all_test_labels[8].reshape(-1,1)
lmot_y_test = all_test_labels[9].reshape(-1,1)
lnos_y_test = all_test_labels[10].reshape(-1,1)
loah_y_test = all_test_labels[11].reshape(-1,1)
msm_y_test = all_test_labels[12].reshape(-1,1)
nii_y_test = all_test_labels[13].reshape(-1,1)
peo_y_test = all_test_labels[14].reshape(-1,1)
rep_y_test = all_test_labels[15].reshape(-1,1)
sm_y_test = all_test_labels[16].reshape(-1,1)
trees_y_test = all_test_labels[17].reshape(-1,1)
v1_y_test = all_test_labels[18].reshape(-1,1)
v2_y_test = all_test_labels[19].reshape(-1,1)

Training the models per superclass

In [ ]:
ne = 100

aq_model = train_single_superclass_model(aq_m_x_train,aq_m_y_train,aq_m_x_test,aq_m_y_test,superclass_names[0],ne)
fish_model = train_single_superclass_model(fish_x_train,fish_y_train,fish_x_test,fish_y_test,superclass_names[1],ne)
flwr_model = train_single_superclass_model(flwr_x_train,flwr_y_train,flwr_x_test,flwr_y_test,superclass_names[2],ne)
fooc_model = train_single_superclass_model(fooc_x_train,fooc_y_train,fooc_x_test,fooc_y_test,superclass_names[3],ne)
fav_model = train_single_superclass_model(fav_x_train,fav_y_train,fav_x_test,fav_y_test,superclass_names[4],ne)
hed_model = train_single_superclass_model(hed_x_train,hed_y_train,hed_x_test,hed_y_test,superclass_names[5],ne)
hf_model = train_single_superclass_model(hf_x_train,hf_y_train,hf_x_test,hf_y_test,superclass_names[6],ne)
ins_model = train_single_superclass_model(ins_x_train,ins_y_train,ins_x_test,ins_y_test,superclass_names[7],ne)
lc_model = train_single_superclass_model(lc_x_train,lc_y_train,lc_x_test,lc_y_test,superclass_names[8],ne)
lmot_model = train_single_superclass_model(lmot_x_train,lmot_y_train,lmot_x_test,lmot_y_test,superclass_names[9],ne)
lnos_model = train_single_superclass_model(lnos_x_train,lnos_y_train,lnos_x_test,lnos_y_test,superclass_names[10],ne)
loah_model = train_single_superclass_model(loah_x_train,loah_y_train,loah_x_test,loah_y_test,superclass_names[11],ne)
msm_model = train_single_superclass_model(msm_x_train,msm_y_train,msm_x_test,msm_y_test,superclass_names[12],ne)
nii_model = train_single_superclass_model(nii_x_train,nii_y_train,nii_x_test,nii_y_test,superclass_names[13],ne)
peo_model = train_single_superclass_model(peo_x_train,peo_y_train,peo_x_test,peo_y_test,superclass_names[14],ne)
rep_model = train_single_superclass_model(rep_x_train,rep_y_train,rep_x_test,rep_y_test,superclass_names[15],ne)
sm_model = train_single_superclass_model(sm_x_train,sm_y_train,sm_x_test,sm_y_test,superclass_names[16],ne)
trees_model = train_single_superclass_model(trees_x_train,trees_y_train,trees_x_test,trees_y_test,superclass_names[17],ne)
v1_model = train_single_superclass_model(v1_x_train,v1_y_train,v1_x_test,v1_y_test,superclass_names[18],ne)
v2_model = train_single_superclass_model(v2_x_train,v2_y_train,v2_x_test,v2_y_test,superclass_names[19],ne)

We have our 20 specialised "expert" models each trained on one superclass.
Now building the "gating" or "routing" model which is a single top-level designer which first decides which superclass the image belongs to and then passes the image to the correct superclass-specefic model for final-grained classification.This is a two-tier approach that will be used to classify each image.

For building the gating model: We will train a single model on the entire CIFAR 100 dataset on the coarse-grained labels(superclasses). We will create a CNN which will have 20 neurons in its outpur layer corrosponding to th2 20 classes.

As the problem of underfitting is faced, we made the model more deeper by increasing the number of layers.

In [4]:
# def build_top_level_model():
#     model = Sequential([
#         # First Convolutional Block
#         Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3),kernel_regularizer=l2(0.001)),
#         MaxPooling2D((2, 2)),
#         # Second Convolutional Block
#         Conv2D(64, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)),
#         MaxPooling2D((2, 2)),
#         # Third Convolutional Block
#         Conv2D(128, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(0.001)),
#         MaxPooling2D((2, 2)),
#         # Flatten the feature maps to feed into the dense layers
#         Flatten(),
#         # Dropout layer to prevent overfitting
#         Dropout(0.5),
#         # Fully Connected Layers
#         Dense(128, activation='relu',kernel_regularizer=l2(0.001)),
#         # Output layer with 20 neurons for the 20 superclasses
#         Dense(len(superclass_names), activation='softmax')
#     ])

#     model.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])

#     return model

def build_top_level_model():
    model = Sequential([
        # First Convolutional Block
        Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3), kernel_regularizer=l2(0.001)),
        MaxPooling2D((2, 2)),

        # Second Convolutional Block
        Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
        MaxPooling2D((2, 2)),

        # Third Convolutional Block
        Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
        MaxPooling2D((2, 2)),

        # Flatten the feature maps
        Flatten(),

        # A dropout layer to help with generalization
        Dropout(0.5),

        # Two Fully Connected Layers
        Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
        Dense(128, activation='relu', kernel_regularizer=l2(0.001)),

        # Output layer with 20 neurons for the 20 superclasses
        Dense(len(superclass_names), activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

fine_to_coarse_labels = {}
for i in range(0, 100):
    n = i // 5
    fine_to_coarse_labels[i] = n

superclass_names = [
    "aquatic mammals", "fish", "flowers", "food containers", "fruit and vegetables",
    "household electrical devices", "household furniture", "insects", "large carnivores",
    "large man-made outdoor things", "large natural outdoor scenes",
    "large omnivores and herbivores", "medium-sized mammals", "non-insect invertebrates",
    "people", "reptiles", "small mammals", "trees", "vehicles 1", "vehicles 2"
]

def get_coarse_grained_data():
    (x_train_fine, y_train_fine), (x_test_fine, y_test_fine) = tf.keras.datasets.cifar100.load_data(label_mode='fine')
    
    # Normalize pixel values to be between 0 and 1
    x_train_fine = x_train_fine / 255.0
    x_test_fine = x_test_fine / 255.0
    
    # Map fine-grained labels to coarse-grained labels
    y_train_coarse = np.array([fine_to_coarse_labels[label[0]] for label in y_train_fine])
    y_test_coarse = np.array([fine_to_coarse_labels[label[0]] for label in y_test_fine])

    # Reshape the label arrays to (x, 1)
    y_train_coarse = y_train_coarse.reshape(-1, 1)
    y_test_coarse = y_test_coarse.reshape(-1, 1)
    
    return (x_train_fine, y_train_coarse), (x_test_fine, y_test_coarse)

In [5]:
(x_train_coarse, y_train_coarse), (x_test_coarse, y_test_coarse) = get_coarse_grained_data()

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Building and training the top-level model

In [6]:

is_gpu_available = check_gpu_status()

print("\n--- Training the top-level classifier on 20 superclasses ---")
if is_gpu_available:
    with tf.device('/GPU:0'):
        top_level_model = build_top_level_model()
        top_level_model.fit(x_train_coarse, y_train_coarse, epochs=600, batch_size=128, verbose=1)
else:
    top_level_model = build_top_level_model()
    top_level_model.fit(x_train_coarse, y_train_coarse, epochs=100, batch_size=64, verbose=1)
    

print("\n--- Evaluating the top-level classifier ---")
loss, accuracy = top_level_model.evaluate(x_test_coarse, y_test_coarse, verbose=1)
print(f"Top-Level Test Loss: {loss:.4f}")
print(f"Top-Level Test Accuracy: {accuracy:.4f}")

Found 2 GPU(s): [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

--- Training the top-level classifier on 20 superclasses ---


I0000 00:00:1758110720.205546      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1758110720.206254      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/600


I0000 00:00:1758110726.519990     106 service.cc:148] XLA service 0x7ef57c00dd80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1758110726.521463     106 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1758110726.521487     106 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1758110726.961735     106 cuda_dnn.cc:529] Loaded cuDNN version 90300


 16/391 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.0637 - loss: 3.8094

I0000 00:00:1758110731.574245     106 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


391/391 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - accuracy: 0.0734 - loss: 3.1987
Epoch 2/600
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.1169 - loss: 2.8903
Epoch 3/600
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.1498 - loss: 2.8260
Epoch 4/600
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.1821 - loss: 2.7661
Epoch 5/600
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.2074 - loss: 2.7102
Epoch 6/600
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.2296 - loss: 2.6628
Epoch 7/600
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.2396 - loss: 2.6253
Epoch 8/600
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.2611 - loss: 2.5871
Epoch 9/600
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.2753 - loss: 2.5572
Epoch 10/600
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.2854 - loss: 2.5267
Epoch 11/600
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.2958 - loss: 2.5068
Epoch 12/600
391/391 ━━━━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 

Now there is one top level trained on the 20 superclasses and 20 different models per superclass each trained on the sub classes within the superclasses.

Now all these trained models are combined into a single cohesive hierarchial pipeline.The process is:
1)Superclass prediction: Given a image, the image is fisr passed to the top level model which predicts which superclass the image belongs to.
2)Fine-grained prediction: Based on the superclass prediction, the image is passed to the appropriate specialised superclass model where the fine grained prediction regarding the subclass to which the image belongs will be made.

In [ ]:
fine_grained_classes = ["beaver","dolphin","otter","seal","whale","aquarium fish","flatfish","ray","shark","trout",
                       "orchids","poppies","roses","sunflowers","tulips","bottles","bowls","cans","cups","plates","apples",
                       "mushrooms","oranges","pears","sweet peppers","clock","computer keyboard","lamp","telephone","television",
                       "bed","chair","couch","table","wardrobe","bee","beetle","butterfly","caterpillar","cockroach","bear","leopard","lion",
                       "tiger","wolf","bridge","castle","house","rpad","skyscraper","cloud","forest","mountain","plain","sea","camel","castle","chimpanzee",
                       "elephant","kangaroo","fox","porcupine","possum","raccoon","shrunk","crab","lobster","snail","spider","worm","baby","boy","girl",
                       "man","woman","crocodile","dinosaur","lizard","snake","turtle","hamster","mouse","rabbit","shrew","squirrel","maple","oak","palm","pine",
                       "willow","bicycle","bus","motorcycle","pickup truck","train","lawn-mower","rocket","streetcar","tank","tractor"]

superclass_names = [
    "aquatic mammals", "fish", "flowers", "food containers", "fruit and vegetables",
    "household electrical devices", "household furniture", "insects", "large carnivores",
    "large man-made outdoor things", "large natural outdoor scenes",
    "large omnivores and herbivores", "medium-sized mammals", "non-insect invertebrates",
    "people", "reptiles", "small mammals", "trees", "vehicles 1", "vehicles 2"
]

superclass_models_dict = [aq_model,fish_model,flwr_model,fooc_model,fav_model,hed_model,hf_model,ins_model,lc_model,lmot_model,
                         lnos_model,loah_model,msm_model,nii_model,peo_model,rep_model,sm_model,trees_model,v1_model,v2_model]

Function to take the image url as input and provide the corrosponding image array with shape (32,32,3) as output.This shape is required since all the models are trained on images of shapes (32,32,3)

In [ ]:
def preprocess_image_from_url(url):
   # Create a request with a User-Agent header to mimic a web browser and avoid 403 errors
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    req = urllib.request.Request(url, headers=headers)

    try:
        with urllib.request.urlopen(req) as response:
            image_data = response.read()

        img = Image.open(io.BytesIO(image_data))
        
        img = img.resize((32, 32))
        
        img_array = np.array(img)
        
        if len(img_array.shape) == 2:
            img_array = np.stack([img_array] * 3, axis=-1)
        
        normalized_img_array = img_array / 255.0

        print("Image successfully downloaded and preprocessed.")
        return normalized_img_array

    except urllib.error.HTTPError as e:
        print(f"Error: {e.code} - {e.reason}")
        print("Could not download the image. Please check the URL and try again.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None


In [ ]:
url = "https://images.pexels.com/photos/170811/pexels-photo-170811.jpeg"
img = preprocess_image_from_url(url)

Function which given the image array gives the superclass to which it belongs

In [ ]:
def predict_image_MOE(image, top_level_model, superclass_names,fine_grained_classes,superclass_models_dict):
    image = np.expand_dims(image, axis=0)
    superclass_prediction = top_level_model.predict(image, verbose=0)
    print(superclass_prediction)
    superclass_index = np.argmax(superclass_prediction)
    print(superclass_index)
    predicted_superclass_name = superclass_names[superclass_index]
    print(f"Predicted superclass is {predicted_superclass_name}")
    finegrained_model = superclass_models_dict[superclass_index]
    finegrained_prediction = finegrained_model.predict(image,verbose= 0)
    print(finegrained_prediction)
    finegrain_index = np.argmax(finegrained_prediction)
    print(finegrain_index)
    fin_index = (superclass_index * 5) + finegrain_index
    fin_predicted_class = fine_grained_classes[fin_index]
    print(f"Predicted fine grained class is {fin_predicted_class}")
    
    

In [ ]:
predict_image_MOE(img,top_level_model,superclass_names,fine_grained_classes,superclass_models_dict)